# GPT를 활용해 리뷰 요약 BEST PROMPT 작성
- 서비스에서 사용 할 GPT 모델: GPT 3.5-TURBO 

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# 환경변수 로딩
load_dotenv('../.env', override=True)           # 인식이 안된다면 ../  > 붙이세연 상위에 있다는 뜻

# 메모리에 로딩된 값을 api_key 변수에 대입
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


# 데이터 전처리
- 리뷰데이터 파일 로딩
- 리뷰 요약 기준 (평점 1~5)
    - good: 평점 5
    - bad: 평점 4 이하
- 각 리뷰마다 special token
    - '[REVIEW_START]' + ['review'] + '[REVIEW_END]'
- 리뷰를 하나의 문자열로 통합
    - reviews_good, reviews_bad



In [3]:
import json

with open('./res/3.kyongpo_the_hongc.json', 'r') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다',
  'stars': 5,
  'date': '2025.02.03'},
 {'review': '로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다',
  'stars': 5,
  'date': '2024.12.23'},
 {'review': '친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요',
  'stars': 5,
  'date': '2024.12.09'}]

In [4]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(329, 91)

In [5]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        # 스페셜 토큰 추가
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_good[:3]

['[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]',
 '[REVIEW_START]로비도 고급지고 객실도 깔끔하니 너무나 만족스런1박 했습니다.\n침구가 너무 좋아서 잘 잘수있었어요.\n다음에 여행가면 여기에 다시 묵을겁니다[REVIEW_END]',
 '[REVIEW_START]친절한 안내와 정리정돈도 잘 되어 있어 편안하게 잘수있었어요[REVIEW_END]']

In [6]:
# GOOD 문자, 하나의 문자열로 합치기 / .join
reviews_good_text = '\n'.join(reviews_good)
reviews_good_text[:100]

'[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]\n[REVIEW_START]로'

In [7]:
# bad 문자, 하나의 문자열로 합치기
reviews_bad_text = '\n'.join(reviews_bad)
reviews_bad_text[:100]

'[REVIEW_START]방음이 좀 안되긴 한데 욕실 깨끗해서 좋아요[REVIEW_END]\n[REVIEW_START]공사한다고 예약힐때부터 알려주면좋겠고 체크인할때 공사한다고 불편'

In [8]:
import datetime
current_datetime = datetime.datetime.now()
date_boundary = current_datetime - datetime.timedelta(days=6*30)

- 하나의 숙박시설에 대한 리뷰데이터 전처리 함수 정의

In [9]:
# import json
# from dateutil import parser

# # 정의부
# def preprocess_reviews(data_path) : 
#     with open(data_path, 'r') as f:
#         review_list = json.load(f)
#     # print(review_list)

#     reviews_good, reviews_bad = [], []

#     for r in review_list:
        
#         if r['stars'] == 5:
#             # 스페셜 토큰 추가
#             reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
#         else:
#             reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

#     reviews_good_text = '\n'.join(reviews_good)
#     reviews_bad_text = '\n'.join(reviews_bad)

#     return reviews_good_text, reviews_bad_text

# # print(reviews_good)
# print(reviews_bad)

In [10]:
# # 호출부
# data_path = "res/1.gangneung_mare.json"
# preprocess_reviews(data_path)

In [11]:
import datetime
import json
from dateutil import parser

def preprocess_reviews(path):
    with open(path, 'r', encoding='utf-8') as f:
        # list 형태로 json 파일 일기
        review_list = json.load(f)

    # good, bad 데이터 저장 변수 초기화
    reviews_good, reviews_bad = [], []

    # 현재 이전 6개월 데이터 사용을 위해 날짜계산
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    # print(date_boundary)

    # 리뷰데이터 
    for r in review_list:
        review_date_str = r['date']
        try:
            # 리뷰 날짜 문자열을 datetime 객체로 변환
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 정상 날짜 데이터 아닌경우 현재 날짜 대입
            review_date = current_date

        # 날짜 바운더리 보다 작으면, pass
        if review_date < date_boundary:
            continue

        # 평점 5점 기준
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # good, bad 하나의 문자열 리스트로 합치기
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    # good 리뷰와 bad 리뷰 반환
    return reviews_good_text, reviews_bad_text

# 리뷰 데이터 전처리 함수 호출
data='./res/3.kyongpo_the_hongc.json'
good, bad = preprocess_reviews(data)
good[:100]

'[REVIEW_START]강릉 여행길에  들럿는데  아주 예약을 잘했어요\n호텔이  주인의 품격이 느껴지더군요  \n조용하고 좋습니다[REVIEW_END]\n[REVIEW_START]로'

# 평가용 함수 작성

## 평가 기준 설정

- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
    - 비교하는 방식 vs. 점수 매기는 방식
    - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
    - 경험상 점수보다는 비교가 상대적으로 더 정확한 편
## 평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)
    - [[A]] – 어시스턴트 A가 더 나을 경우
    - [[B]] – 어시스턴트 B가 더 나을 경우
    - [[C]] – 두 응답이 동등한 품질일 경우

In [12]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
공정한 심사위원으로서 두 분이 제공한 한국어 요약의 품질을 평가해 주시기 바랍니다
아래에 표시된 숙소에 대한 사용자 리뷰 세트의 AI 어시스턴트. 다음과 같은 어시스턴트를 선택해야 합니다
사용자의 지시를 따르고 사용자의 질문에 더 잘 답합니다. 당신의 평가
유용성, 관련성, 정확성, 깊이, 창의성과 같은 요소들을 고려해야 합니다,
응답의 세부 사항 수준. 두 가지를 비교하여 평가를 시작합니다
답변을 제공하고 간단한 설명을 제공합니다. 위치 편향을 피하고 다음을 보장합니다
응답이 제시된 순서는 결정에 영향을 미치지 않습니다. 허용하지 않음
평가에 영향을 미치는 응답의 길이. 특정 이름을 선호하지 않습니다
보조원들. 가능한 한 객관적으로 행동하세요. 설명을 제공한 후, 다음과 같이 출력하세요
이 형식을 엄격히 따름으로써 최종 판결: "[[A]]" 만약 보조원 A가 더 나은 경우, "[B]"
조수 B가 더 나은 경우, 그리고 동점일 경우 "[C]".
[User Reviews]
{reviews}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""
    
        # OpenAI API를 사용하여 평가 요청
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion.choices[0].message.content

## BASELINE 응답개발

In [13]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [14]:
reviews, _ = preprocess_reviews(path='./res/3.kyongpo_the_hongc.json')  #, _ 두번째 응답은 안받음

## 리뷰데이터 요약 함수 정의 및 실행

In [15]:
def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion.choices[0].message.content

print(summarize(reviews, PROMPT_BASELINE))

강릉 여행 중 예약한 호텔은 주인의 품격이 느껴지며 조용하고 좋았다. 로비와 객실이 고급스럽고 깔끔해서 만족스러웠고, 침구가 편안해서 잘 수 있었다. 친절한 안내와 청결한 환경으로 편안하게 머물렀고, 위치도 좋아서 편안한 휴식을 즐겼다. 부모님도 만족하셨고, 직원들의 친절함과 시설의 깨끗함이 좋았다.


## 응답결과 평가하기
- 대조군 문장 정의하기

In [16]:
# 대조군 문장 가져오기(야놀자 AI good 요약 문장)
# summary_real_20250530 = '숙소는 강릉 경포해변 바로 앞에 위치하며, 오션뷰와 경포호수 전망을 제공합니다. 객실과 화장실이 청결하게 관리되고 있으며, 편안한 침대와 고급 침구로 쾌적한 숙박 환경을 제공합니다. 주변에 식당과 편의시설이 가까워 접근성이 좋고, 주차 공간도 넓습니다. 직원들의 친절한 응대와 서비스도 긍정적인 평가를 받고 있으며, 대체로 소음 차단이 잘 이루어져 조용한 환경에서 휴식을 취할 수 있습니다. 전반적으로 방문자들은 숙소의 청결도와 편안함, 아늑한 분위기에 매우 만족하는 후기를 남기고 있습니다.'

summary_4o_20250530 = summarize(reviews, PROMPT_BASELINE, temperature=0.0, model='gpt-4o-2024-05-13')
summary_4o_20250530

'이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용하여 편안하게 쉴 수 있는 곳으로 추천받고 있습니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 긍정적으로 언급되었습니다. 다만, 배달 음식 이용이 제한된다는 점과 방음이 다소 아쉽다는 의견도 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.'

In [17]:
# 비교하기
# pairwise_eval(리뷰 데이터, AI 요약한 데이터_A, 대조군 AI 문장_B))
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE), summary_4o_20250530))

두 보조원의 답변을 비교해 보겠습니다.

### Assistant A
- **유용성**: 리뷰의 주요 포인트를 잘 요약하여 제공하고 있습니다.
- **관련성**: 리뷰의 핵심 내용을 잘 반영하고 있습니다.
- **정확성**: 리뷰에서 언급된 내용을 정확하게 전달하고 있습니다.
- **깊이**: 전반적인 평가와 함께 부모님이 만족하셨다는 점, 직원들의 친절함, 청결함 등을 언급하여 깊이 있는 요약을 제공하고 있습니다.
- **창의성**: 리뷰의 다양한 측면을 잘 조합하여 자연스럽게 요약했습니다.

### Assistant B
- **유용성**: 리뷰의 주요 포인트를 잘 요약하여 제공하고 있습니다.
- **관련성**: 리뷰의 핵심 내용을 잘 반영하고 있습니다.
- **정확성**: 리뷰에서 언급된 내용을 정확하게 전달하고 있습니다.
- **깊이**: 배달 음식 이용 제한과 방음 문제를 언급하여 부정적인 측면도 포함한 점이 좋습니다.
- **창의성**: 긍정적인 측면과 부정적인 측면을 균형 있게 다루었습니다.

### 최종 평가
두 보조원 모두 리뷰의 주요 내용을 잘 요약했지만, Assistant B는 부정적인 측면도 언급하여 더 균형 잡힌 평가를 제공했습니다. 따라서 최종 판결은 다음과 같습니다:

[[B]]


# 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [18]:
# 같은 리뷰에 대해서 요약을 10번 수행(일반적으로는 50~100번 수행하면 좋음)
eval_count = 10


summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0) for _ in range(eval_count)]
summaries_baseline

['강릉 여행에 예약한 호텔은 주인의 품격이 느껴지며 조용하고 좋았다. 객실과 침구가 깔끔하고 좋아서 만족스럽게 1박했다. 친절한 안내와 정리정돈으로 편안하게 묵었고, 위치가 좋고 깨끗해서 잘 쉬고 간다고 했다. 숙소는 깨끗하고 친절하며 이쁘고 좋다는 평가가 많았다. 위치가 좋고 시설이 깔끔하며 친절한 직원들이 있어 편하게 묶었다고 한다.',
 '좋은 서비스와 깨끗한 숙소로 만족스럽게 머물렀다. 위치도 좋고 침대도 편안해서 잘 쉬다 왔다. 또 방음이 조금 부족한 점 빼고는 다른 부분에서 모두 만족스러웠다.',
 '강릉 여행길에 예약한 숙소는 조용하고 주인의 품격을 느낄 수 있는 호텔이었고, 고급스러운 로비와 깔끔한 객실이 만족스러웠습니다. 침구가 편안하고 깨끗한 숙소로, 친절한 안내와 정리정돈으로 편안하게 쉴 수 있었어요. 위치도 좋고 청결한 객실에서 잘 쉬다가 갑니다. 부모님도 만족하셨고, 직원들의 친절함과 시설의 깔끔함을 극찬하는 리뷰가 많이 있었습니다.',
 '강릉 여행길에 머물렀는데 예약을 잘한 호텔로 조용하고 품격 느껴짐. 고급 로비와 깔끔한 객실로 만족하고 다시 방문 의사 있음. 친절한 안내와 청결한 객실에 만족하며 위치도 좋음. 직원 친절과 깔끔한 시설로 편하고 만족스러운 숙박하며 침구 상태도 좋았음. 부모님도 만족하셨고, 위치도 좋고 깨끗하며 친절한 직원에게 감사함.',
 '강릉 여행 중 아주 만족스런 호텔, 주인의 품격 느껴짐. 조용하고 깔끔한 객실, 침구도 좋아서 잠도 편안함. 깨끗하고 위치 좋아서 편안히 쉬었고, 친절한 안내에 만족. 부모님도 만족하셨고, 쾌적한 환경과 친절한 직원에 감동.',
 '이 호텔은 강릉 여행길에 위치해 조용하고 고급스러운 분위기를 느낄 수 있는 곳이며, 깨끗하고 침구가 편안해서 만족스러운 1박을 할 수 있습니다. 친절한 안내와 청결한 시설, 그리고 주변 인프라가 좋아서 휴식을 취하기에 안성맞춤입니다. 또한 직원들의 친절한 서비스와 다양한 편의시설이 제공되어 고객들에게 추천할만한 숙소입니다.',
 '숙소는 예약하기 

In [19]:
pairwise_words = [summary_4o_20250530 for _ in range(len(summaries_baseline))]
pairwise_words

['이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용하여 편안하게 쉴 수 있는 곳으로 추천받고 있습니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 긍정적으로 언급되었습니다. 다만, 배달 음식 이용이 제한된다는 점과 방음이 다소 아쉽다는 의견도 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용하여 편안하게 쉴 수 있는 곳으로 추천받고 있습니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 긍정적으로 언급되었습니다. 다만, 배달 음식 이용이 제한된다는 점과 방음이 다소 아쉽다는 의견도 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용하여 편안하게 쉴 수 있는 곳으로 추천받고 있습니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 긍정적으로 언급되었습니다. 다만, 배달 음식 이용이 제한된다는 점과 방음이 다소 아쉽다는 의견도 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용하여 편안하게 쉴 수 있는 곳으로 추천받고 있습니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 긍정적으로 언급되었습니다. 다만, 배달 음식 이용이 제한된다는 점과 방음이 다소 아쉽다는 의견도 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은 숙소입니다.',
 '이 숙소는 청결하고 침구 상태가 좋으며, 직원들이 친절하다는 평가가 많습니다. 위치가 좋고 조용하여 편안하게 쉴 수 있는 곳으로 추천받고 있습니다. 로비와 객실이 고급스럽고 깔끔하다는 점도 긍정적으로 언급되었습니다. 다만, 배달 음식 이용이 제한된다는 점과 방음이 다소 아쉽다는 의견도 있습니다. 전반적으로 재방문 의사가 많고 만족도가 높은

In [20]:
from tqdm import tqdm  # 진행상태를 보여줌

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        # 최종 결과
        verdict_text = pairwise_eval(reviews, answers_a[i], answers_b[i])

        # A인경우 카운트
        if '[[A]]' in verdict_text:
            a_cnt += 1
        # B인경우 카운트
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        # C인경우 카운트
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        # 나머지
        else:
            print('Evaluation Error')
    # a good, b bad, c 무승부 비슷함.
    return a_cnt, b_cnt, draw_cnt

# 요약한 것과 대조군 비교
wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_4o_20250530 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

 10%|█         | 1/10 [00:04<00:43,  4.83s/it]

Evaluation Error


 30%|███       | 3/10 [00:11<00:25,  3.71s/it]

Evaluation Error


 50%|█████     | 5/10 [00:18<00:17,  3.57s/it]

Evaluation Error


 70%|███████   | 7/10 [00:25<00:10,  3.50s/it]

Evaluation Error


100%|██████████| 10/10 [00:37<00:00,  3.72s/it]

Evaluation Error
Wins: 0, Losses: 5, Ties: 0


## 응답결과 고도화 1 - 요약 조건들 명시 후 평가
- role과 수행 목표 정의
- 요약 결과에 대한 충족 조건
- 톤엔 매너와 예시

In [21]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0) for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

 30%|███       | 3/10 [00:10<00:24,  3.50s/it]

Evaluation Error


 40%|████      | 4/10 [00:13<00:20,  3.43s/it]

Evaluation Error


 80%|████████  | 8/10 [00:27<00:06,  3.25s/it]

Evaluation Error


100%|██████████| 10/10 [00:35<00:00,  3.59s/it]

Wins: 4, Losses: 3, Ties: 0


## 응답결과 고도화 2 - 입력 데이터의 품질 증가
- 리뷰 길이가 짧은 것 필터링함, 고품질 리뷰만 남김

In [24]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/3.kyongpo_the_hongc.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        # 고품질 리뷰만 남김
        if len(r['review']) < 30:
            filtered_cnt += 1
            # print(r['review'])
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    # 최대 길이가 50까지 되도로 자름 
    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

## 응답결과 고도화 3 - One-Shot Prompting

In [28]:
reviews_1shot, _ = preprocess_reviews('./res/3.kyongpo_the_hongc.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-3.5-turbo-0125')
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:31<00:00,  3.12s/it]

Wins: 7, Losses: 3, Ties: 0


In [ ]:
print(summaries)

In [29]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125') for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_4o_20250530 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:26<00:00,  2.61s/it]

Wins: 5, Losses: 5, Ties: 0


## prompt_1shot을 피클파일로 저장하기

In [30]:
# prompt_1shot을 피클파일로 저장하기
file_name = "./res/prompt_1shotv1.pickle"
# 파일로 저장
import pickle
with open(file_name, 'wb') as f:
    pickle.dump(prompt_1shot, f)
    print(summaries)

['강릉 여행길에 예약한 호텔은 주인의 품격을 느낄 수 있는 조용하고 좋은 숙소였습니다. 로비와 객실이 고급스럽고 깔끔하여 만족스러운 1박을 했으며, 침구가 너무 좋아서 편안하게 잘 수 있었습니다. 다음에 여행을 가면 다시 묵을 의향이 있습니다. 또한, 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 숙박을 할 수 있었고, 부모님도 만족하셨습니다. 숙소는 깨끗하고 아늑하며, 직원들도 친절하고 좋았습니다.', '강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었으며, 주인의 품격이 느껴지고 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔하여 너무 만족스러운 1박을 했으며, 침구가 훌륭해서 편안하게 잘 수 있었고 다음에 여행을 가면 다시 묵을 의향이 있습니다. 또한, 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 잘 수 있었고, 부모님도 만족하셨고 깨끗하고 아늑한 숙소이며 객실이 깨끗하고 편안하고 직원분들도 친절하셨습니다.', '강릉 여행길에 들렀을 때 예약을 잘한 호텔이었고, 주인의 품격이 느껴졌으며 조용하고 좋은 숙소였습니다. 로비와 객실이 고급스럽고 깔끔하여 너무 만족스러운 1박을 했고, 침구가 훌륭하여 편안하게 잘 수 있었습니다. 또한, 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 머물렀습니다. 부모님도 만족하셨고, 숙소는 매우 깨끗하고 아늑했습니다. 직원들도 친절했고, 편안한 휴식을 즐길 수 있었습니다.', '강릉 여행길에 들렀을 때 아주 예약을 잘한 호텔이었습니다. 호텔 주인의 품격이 느껴지고 조용하고 좋은 숙소였습니다. 로비도 고급스럽고 객실도 깔끔하여 너무 만족스러운 1박을 했습니다. 다음에 여행을 가면 다시 묵을 의향이 있고, 침구가 훌륭하여 편안하게 잘 수 있었습니다. 친절한 안내와 정리정돈이 잘 되어 있어 편안하게 잘 수 있었고, 부모님도 만족하셨습니다. 또한, 깨끗하고 아늑한 숙소에 자주 갔다가 좋아하며, 객실이 깨끗하고 편안하고 직원분들도 친절해서 좋았습니다.', '강릉 여행길에 들렀을 때 호텔 예약을 잘한 것으로 만족하며 